## Project 1: Class Scheduling with Mathematical Programming

* **Collaborators**: Isidore Sossa and Chase Scott
* **Resources**: 
    * Nurses Scheduling Problem
* **Project's requirements**:
    * Use data provided through Canvas
    * Generate random list of preferences for each professor according to their qualification, i.e. an integer from 1 to *n*, where *n* is the number of classes the professor is qualified to teach. The higher the number the lesser the preference. (**Done**)
    * Extract out professors who do not teach
    * Classes start at 8:00 AM and end at 6:00 PM and can only start at every half period.
    * One credit hour is equivalent to 50 minutes. E.g. A 3-credit hour class meet for a total of 150 minutes per week.
    * Add two constraints of your chosen:
        * No professors teach two courses back-to-back


In [ ]:
import pandas as pd
from docplex.mp.model import Model
import math
import random
import numpy as np
import itertools

### Data Exploration and Cleaning

In [ ]:
## Load data
data_file = 'Data Set – Class Schedule.xlsx'
file_handler = pd.ExcelFile(path_or_buffer=data_file)
file_sheets = sorted(file_handler.sheet_names)
print(f'Sheets : {file_sheets}')

## Load each sheet as a DataFrame
classrooms_df = pd.read_excel(io=file_handler, sheet_name=file_sheets[0])
display(file_sheets[0], classrooms_df)

colleges_and_depts = pd.read_excel(io=file_handler, sheet_name=file_sheets[1])
display(file_sheets[1], colleges_and_depts.head(5))

course_catalog_df = pd.read_excel(io=file_handler, sheet_name=file_sheets[2])
display(file_sheets[2], course_catalog_df.head(5))

courses_offered = pd.read_excel(io=file_handler, sheet_name=file_sheets[3])
display(file_sheets[3], courses_offered.head(45))

professors_df = pd.read_excel(io=file_handler, sheet_name=file_sheets[4])
display(file_sheets[4], professors_df.head(5))

qualifications_df = pd.read_excel(io=file_handler, sheet_name=file_sheets[5])
display(file_sheets[5], qualifications_df.head(5))

# For reproducibility
random.seed(1345)

### Functions and Customizations

In [ ]:
## Duplicate courses with multiple sessions
def standardize_courses_offered(courses_offered:pd.DataFrame, course_catalog_df:pd.DataFrame) -> pd.DataFrame:
    ''''''
    def get_day_name(day:str):
        days_name = {'M':'Monday', 'TU':'Tuesday', 'W':'Wednesday', 'TH':'Thursday', 'F':'Friday'}
        return days_name[day]
    
    result = pd.DataFrame()
    result = result.append(courses_offered[courses_offered['Sections'] == 1])
    for course in courses_offered[courses_offered['Sections'] > 1].itertuples():
        course = pd.DataFrame(course).T.drop(columns=0)
        course.columns = courses_offered.columns
        for i in range(1, course['Sections'][0] + 1):
            course['Sections'] = i
            result = result.append([course], ignore_index=True)
    result = result.rename(columns={'Sections' : 'Session'})
    result = result.sort_values(by=['Course Number', 'Session']).reset_index().drop(columns='index')
    result = pd.merge(left=result, right=course_catalog_df, on=['Department Code', 'Course Number', 'Course Name'])
    
    courses_offered = result
    result = pd.DataFrame()
    
    for course in courses_offered.itertuples():
        course = pd.DataFrame(course).T.drop(columns=0)
        course.columns = courses_offered.columns
        days_taught = course['Day'][0]
        days_taught = days_taught.split('/')
        for i in days_taught:
            course['Day'] = i
            result = result.append([course], ignore_index=True)
            
    result['Day'] = result.apply(lambda row : get_day_name(row.Day), axis=1)
    return result

courses_offered = standardize_courses_offered(courses_offered, course_catalog_df)

In [ ]:
## Clean
def merge_and_get_preference(professors_df:pd.DataFrame, qualifications:pd.DataFrame, course:str) -> pd.DataFrame:
    ''''''
    def manipulate_row(row:pd.DataFrame, course:str):
        cols = [x for x in row.columns if course in x]
        class_taught = 0
        for col in cols:
            if not math.isnan(row[col][0]):
                class_taught += row[col][0]
        class_taught = int(class_taught)
        if row['Workload Credit Hours'][0] > 0:
            preferences = random.sample(list(range(1, class_taught + 1)), class_taught)
        else:
            preferences = [0] * class_taught
            
        #display(preferences)

        index = 0
        for col in cols:
            if not math.isnan(row[col][0]):
                row[col] = preferences[index]
                index += 1
            else:
                row[col] = 0
        return row

    result_df = pd.merge(left=professors_df, right=qualifications_df, on=['Faculty Name', 'Department Code'])
    col_names = list(result_df.columns)
    preferences_df = pd.DataFrame(columns=col_names)
    col_names = result_df.columns
    for row in result_df.itertuples():
        row_df = pd.Series(row).to_frame().T.drop(columns=0)
        row_df.columns = col_names
        row_df = manipulate_row(row_df, course)
        preferences_df = preferences_df.append(row_df, ignore_index=True)
    return preferences_df

preferences_df = merge_and_get_preference(professors_df, qualifications_df, 'ENTOM')
display(preferences_df)

In [ ]:
def get_period_df() -> pd.DataFrame:
    '''Create a dataframe with days'''
    days = ['Monday', 'Tuesday', 'Wednesday', 'Thrusday', 'Friday']
    days_of_weeks = dict(zip(days, range(1, 6)))
    days_of_weeks

    result = pd.DataFrame(columns=['Period', 'Day'])
    start = 1
    day_digits = []
    day_names = []
    for day, day_digit in days_of_weeks.items():
        day_names.extend([day] * 20)
        day_digits.extend([day_digit] * 20)
        
    start_hour = 8
    end_hour = 18
    start_time = [x * 0.5 for x in range(2*start_hour, 2*end_hour)] * len(days)
    end_time = [x * 0.5 for x in range(2*start_hour + 1, 2*end_hour + 1)] * len(days)
    
    
    return pd.DataFrame.from_dict({'Period' : list(range(1, 21)) * 5, 'Day Name' : day_names, 'Day Number' : day_digits, 
                                   'Start Time' : start_time, 'End Time' : end_time})

period_df = get_period_df()

In [ ]:
def get_column_value_as_list(df:pd.DataFrame, colname:str) -> pd.DataFrame:
    '''Get the values of a data frame column as a Python list'''
    if colname in df.columns:
        return df[colname].values.tolist()
    else:
        return None

### Mathematical Model

In [ ]:
## Model Environment
model = Model(name='Course Scheduling')

######################
# Decision Variables #
######################
x_ijklmn = dict()
for professor in professors_df['Faculty Name']:
    for course in courses_offered.groupby(by=['Course Name', 'Session', 'Day']):
        course_name, session, day_offered = course[0]
        for period in np.unique(get_column_value_as_list(period_df, 'Period')):
            for room in classrooms_df['Classroom']:
                x_ijklmn[(professor, course_name, session, room, period, day_offered)] = model.binary_var(name = "x({},{},{},{},{},{})".format(professor, course_name, session, room, period, day_offered))

model.print_information()

In [ ]:
######################
#     Constraints    #
######################

# Constraint 1: Each course is assigned to only one professor in only one room
for course in courses_offered.groupby(by=['Course Name', 'Session', 'Day']):
    course_name, session, day_offered = course[0]
    for period in period_df['Period']:
        aggregate_values = []
        for professor in professors_df['Faculty Name']:
            for room in classrooms_df['Classroom']:
                aggregate_values.append(x_ijklmn[(professor, course_name, session, room, period, day_offered)])
        model.add_constraint(model.sum(aggregate_values) == 1)
    
# Constraint 2: Courses last the same period on the days they are offered
for course in courses_offered.groupby(by='Course Name'):
    CLASS_DURATION = 50
    PERIOD_DURATION = 30
    course_name = course[0]
    course_df = courses_offered[courses_offered['Course Name'] == course_name]
    for session in course_df.groupby(by='Session'):
        course_session_df = course_df[course_df['Session'] == session[0]]
        course_session_value = course_session_df['Session'].values.tolist()[0]
        number_of_credit = np.unique(course_session_df['Credit Hours'].values.tolist())[0]
        duration = (number_of_credit * CLASS_DURATION) / (len(course_session_df) * PERIOD_DURATION)
        number_of_period_covered = math.ceil(duration)
        
        for day_offered in course_session_df.groupby(by='Day'):
            day_offered_df = course_session_df[course_session_df['Day'] == day_offered[0]]
            day_period_df = period_df[period_df['Day Name'] == day_offered[0]]
            day_value = day_offered_df['Day'].values.tolist()[0]
            period_list = day_period_df['Period'].values.tolist()
            for index, _ in enumerate(period_list[0:len(period_list)-number_of_period_covered]):
                sessions = period_list[index:index+number_of_period_covered]
                aggregate_values = []
                for period in sessions:
                    for professor in professors_df['Faculty Name']:
                        for room in classrooms_df['Classroom']:
                            aggregate_values.append(x_ijklmn[(professor, course_name, course_session_value, room, period, day_value)])
                model.add_constraint(model.sum(aggregate_values) == number_of_period_covered)
                
# Constraint 3: Courses are offered at the same period across the days they are offered
for course in courses_offered.groupby(by=['Course Name', 'Session']):
    course_name, session = course[0]
    course_df = courses_offered[courses_offered['Course Name'] == course_name]
    course_df = course_df[course_df['Session'] == session]
    for professor in professors_df['Faculty Name']:
        for room in classrooms_df['Classroom']:
            for period in np.unique(get_column_value_as_list(period_df, 'Period')):
                aggregate_values = []
                for days_pair in itertools.combinations(course_df['Day'].values.tolist(), 2):
                    for day_offered in days_pair:
                        aggregate_values.append(x_ijklmn[(professor, course_name, session, room, period, day_offered)])
                model.add_constraint(aggregate_values[0] - aggregate_values[1] == 0)

                
# Constraint 4: One the days they are offered, courses are taught in a classroom that has at the maximum enrollment capacity
for course in courses_offered.groupby(by=['Course Name', 'Session', 'Day', 'Max Enrollement']):
    course_name, session, day_offered, max_enrollement = course[0]
    for professor in professors_df['Faculty Name']:
        for room in classrooms_df.groupby(by=['Classroom', 'Capacity']):
            classroom, capacity = room[0]
            for period in np.unique(get_column_value_as_list(period_df, 'Period')):
                model.add_constraint(x_ijklmn[(professor, course_name, session, classroom, period, day_offered)] * max_enrollement <= capacity)
                
# Constraint 5: A professor can only teach one course per period on the day the course is offered 
# among all the courses that are offered that day
for professor in professors_df['Faculty Name']:
    for period in np.unique(get_column_value_as_list(period_df, 'Period')):
        for course in courses_offered.groupby(by='Day'):
            aggregate_values = []
            day_offered_df = courses_offered[courses_offered['Day'] == course[0]]
            display(day_offered_df)
            for room in classrooms_df['Classroom']:
                for course_name in day_offered_df.itertuples():
                    course_df = pd.DataFrame(course_name).T.drop(columns=0)
                    course_df.columns = day_offered_df.columns
                    aggregate_values.append(x_ijklmn[(professor, course_df['Course Name'][0], course_df['Session'][0], 
                                                      room, period, course_df['Day'][0])])
            model.add_constraint(model.sum(aggregate_values) <= 1)

    #Need
        #Capacity
        #Time start no earlier than 0800
        #Time end no later than 1800
        #Every course must be assigned
        #no section  can  be  assigned  to  more  than  one  professor

    #Rule 1
        #Prof doesn't have back to back course
            #Do by sum of consecutive periods with a break of some arbitrary number
    #Rule 2
        #No scheduled classes after noon on Friday

In [ ]:
######################
# Objective Function #
######################
aggregate_values = []
for professor in preferences_df.itertuples():
    professor_df = pd.DataFrame(professor).T.drop(columns=0)
    professor_df.columns = preferences_df.columns
    professor_name = preferences_df['Faculty Name'][0]
    for course in courses_offered.itertuples():
        course_df = pd.DataFrame(course).T.drop(columns=0)
        course_df.columns = courses_offered.columns
        course_code = '{} {}'.format(course_df['Department Code'][0], str(course_df['Course Number'][0]))
        course_name = '{}'.format(course_df['Course Name'][0])
        course_session = course_df['Session'][0]
        course_day_offered = course_df['Day'][0]
        course_preference = preferences_df[course_code][0]
        for period in np.unique(get_column_value_as_list(period_df, 'Period')):
            for room in classrooms_df['Classroom']:
                aggregate_values.append(course_preference * x_ijklmn[(professor_name, course_name, course_session, room, period, course_day_offered)])

objective_function = sum(aggregate_values)
model.maximize(objective_function)

In [ ]:
######################
# Objective Function #
######################

model.print_information()
solution = model.solve(log_output = True, time_limit = 300)
assert solution, "Solve Failed"
model.report()

model.print_solution()

In [ ]:
courses_offered

In [ ]:
display(aggregate_values)
courses_offered.sort_values(by='Course Name')

In [ ]:
display(classrooms_df)
for professor in professors_df['Faculty Name']:
    print(professor)

In [ ]:
for course in courses_offered.groupby(by=['Course Name', 'Session', 'Day']):
    print(course[0])

In [ ]:
courses_offered